# プーリング層の実装
プーリング層の具体的な実装方法を解説します。  
順伝播、逆伝播の実装をそれぞれコードを交えて解説します。

## ●プーリング層のクラス
今回はMAXプーリングを行います。  
小さく区切られた各領域から最大値を取り出し、サイズが小さくなった新たな画像を作ります。  

プーリング層は、以下のようにクラスとして実装します。  

In [ ]:
# -- プーリング層 --
class PoolingLayer:
    
    # n_bt:バッチサイズ, x_ch:入力チャンネル数, x_h:入力画像高さ, x_w:入力画像幅
    # pool:プーリング領域のサイズ, pad:パディング幅
    # y_ch:出力チャンネル数, y_h:出力高さ, y_w:出力幅
    
    def __init__(self, x_ch, x_h, x_w, pool, pad):
        
        # パラメータをまとめる
        self.params = (x_ch, x_h, x_w, pool, pad)
        
        # 出力画像のサイズ
        self.y_ch = x_ch  # 出力チャンネル数
        self.y_h = x_h//pool if x_h%pool==0 else x_h//pool+1  # 出力高さ
        self.y_w = x_w//pool if x_w%pool==0 else x_w//pool+1  # 出力幅

        ...

__init__メソッドでは、変更したり外部からアクセスしたりする必要のないパラメータを`self.params`にまとめています。  
また、出力画像のチャンネル数、高さ、幅は、外部からアクセス可能にするために`self`が付けられています。  
このクラスに、順伝播と逆伝播のメソッドを記述します。

## ●プーリング層の順伝播
プーリング層における順伝播ではまず、im2colにより画像を行列に変換します。  
そして、この行列の各列の最大値から画像を再構成します。  
これを以下の図に示します。

<img src="images/pooling_forward_image.png">


これにより、各領域の最大値が抽出された画像が生成されます。  

この流れを別の図で表すと、以下のようになります。

<img src="images/pooling_forward.png">

この図を踏まえて、プーリング層における順伝播を以下のようにメソッドとして実装します。

In [ ]:
    def forward(self, x):
        n_bt = x.shape[0] 
        x_ch, x_h, x_w, pool, pad = self.params
        y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
        
        # 入力画像を行列に変換
        cols = im2col(x, pool, pool, y_h, y_w, pool, pad)
        cols = cols.T.reshape(n_bt*y_h*y_w*x_ch, pool*pool)
        
        # 出力の計算: Maxプーリング
        y = np.max(cols, axis=1)
        self.y = y.reshape(n_bt, y_h, y_w, x_ch).transpose(0, 3, 1, 2)
        
        # 最大値のインデックスを保存
        self.max_index = np.argmax(cols, axis=1)

入力画像の形式は、畳み込み層のものと同じです。  
まずは、im2colにより入力画像を行列に変換します。  

フィルタの幅と高さ、ストライドの幅にはプーリング領域のサイズを指定します。  
これにより、入力画像を正方形の領域に区切り、それを行列の各列にすることができます。  
得られた行列`cols`は、転置と`reshape`により最大値をとるための形状に変換します。
  
次に、NumPyのmax関数を使って、行列`cols`の各行で最大値を求めます。  
これによりもとめた`y`は、`cols`から次元が1つ減ることになります。  
`y`は、形状が整えられた上で出力`self.y`となります。  

最後に、各列の最大値のインデックスを、後に逆伝播で利用するため`self.max_index`に保存しておきます。  
NumPyのargmax関数は、最大値そのもではなくて最大値のインデックスを取得します。  

## ●プーリング層の逆伝播
プーリング層でも、逆伝播では入力の勾配を伝播させます。  
プーリング層の逆伝播では、各領域の最大値のピクセルのみで誤差が伝播します。  
そのため、順伝播の際に保存しておいた、各領域における最大値のインデックスを利用した少々トリッキーなアルゴリズムが必要になります。  

このアルゴリズムではまず、順伝播の際に各列の最大値を取得した行列と、同じ形状の行列を作ります。  
そして、この行列における各列の、最大値があった要素に、出力の勾配を格納します。  
これを以下図に示します。

<img src="images/pooling_backward_image_1.png">

各列の最大値であった要素に出力の勾配を入れて、残りは0にします。

そして、これをcol2imにより画像に復元し、入力の勾配とします。 
これを以下の図に示します。  

<img src="images/pooling_backward_image_2.png">

プーリング層では学習は行われないので、重みやバイアスのようなものはありません。  
従って、逆伝播で求める必要があるのは入力の勾配のみです。    

プーリング層の逆伝播を図で表すと、以下の図のようになります。

<img src="images/pooling_backward.png">

この図を踏まえて、プーリング層における逆伝播を以下のようにメソッドして実装します。

In [ ]:
    def backward(self, grad_y):
        n_bt = grad_y.shape[0] 
        x_ch, x_h, x_w, pool, pad = self.params
        y_ch, y_h, y_w = self.y_ch, self.y_h, self.y_w
        
        # 出力の勾配の軸を入れ替え
        grad_y = grad_y.transpose(0, 2, 3, 1)
        
        # 行列を作成し、各列の最大値であった要素にのみ出力の勾配を入れる
        grad_cols = np.zeros((pool*pool, grad_y.size))
        grad_cols[self.max_index.reshape(-1), np.arange(grad_y.size)] = grad_y.reshape(-1) 
        grad_cols = grad_cols.reshape(pool, pool, n_bt, y_h, y_w, y_ch)
        grad_cols = grad_cols.transpose(5,0,1,2,3,4) 
        grad_cols = grad_cols.reshape( y_ch*pool*pool, n_bt*y_h*y_w)

        # 入力の勾配
        x_shape = (n_bt, x_ch, x_h, x_w)
        self.grad_x = col2im(grad_cols, x_shape, pool, pool, y_h, y_w, pool, pad)

引数として出力の勾配`grad_y`を受け取りますが、この軸をまず`transpose`で入れ替えます。  
そして、Numpyのzeros関数により、全ての要素が0の行列`grad_cols`を作成します。
 
行列`grad_cols`の行数は、`pool*pool,`、すなわちプーリング領域の全要素数とします。  
また、列数は`grad_y.size`、すなわち`grad_y`の全要素数になります。

次に、この行列に`grad_y`を当てはめます。  
当てはめる位置は、順伝播の際に保存した`self.max_index`の位置とします。  

`self.max_index.reshape(-1)`ではまず、`rehape(-1)`により一次元配列に変換します。  
これを、当てはめる位置のインデックスとします。
`np.arange(grad_y.size)`で0から`grad_y.size-1`までの列のインデックスを指定します。  
指定された各インデックスに、出力の勾配を1次元配列にしたもの`grad_y.reshape(-1)`を当てはめます。  
そして、`grad_cols`の形状をtransposeとreshapeで整えて、一つの行列にします。

以上により、col2imで変換が可能な形状の行列を得ることができました。  
これを先ほど図で示したようにcol2imで画像に戻して、入力の勾配`self.grad_x`とします。

プーリング層における逆伝播では、重みの勾配とバイアスの勾配を求める必要が無いので、逆伝播における処理は以上になります。